<a href="https://colab.research.google.com/github/Alyona18/text-classification-with-transformers/blob/main/RuBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp  drive/MyDrive/НИР/dataset_bert.py .
!cp  drive/MyDrive/НИР/functions_for_bert.py .

In [ ]:
!pip install transformers
!pip install torch
!pip3 install deeppavlov
import torch
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
!cp  drive/MyDrive/НИР/dataset_bert.py .
!cp  drive/MyDrive/НИР/functions_for_bert.py .

from dataset_bert import CustomDataset
from functions_for_bert import Classifier

In [ ]:
from transformers import BertTokenizer
device = 'cuda'
tokenizer_path = 'cointegrated/rubert-tiny'
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


In [ ]:
from transformers import BertForSequenceClassification
model_path = 'cointegrated/rubert-tiny'
model = BertForSequenceClassification.from_pretrained(model_path)

/usr/local/lib/python3.7/dist-packages/cryptography/hazmat/backends/openssl/x509.py:17: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSeque

In [ ]:
out_features = model.bert.encoder.layer[1].output.dense.out_features

In [ ]:
classifier_RB = Classifier(
    model,
    tokenizer,
    out_features,
    6
)

In [ ]:
from deeppavlov.dataset_readers.basic_classification_reader import BasicClassificationDatasetReader
from deeppavlov.dataset_iterators.basic_classification_iterator import BasicClassificationDatasetIterator

# read data from particular columns of `.csv` file
dr = BasicClassificationDatasetReader().read(
    data_path='/content/drive/MyDrive/НИР',
    train='Join_dataset.csv',
    x = 'text',
    y = 'news'
)

# initialize data iterator splitting `train` field to `train` and `valid` in proportion 0.8/0.2
train_iterator = BasicClassificationDatasetIterator(
    data=dr,
    field_to_split='train',  # field that will be splitted
    split_fields=['train', 'valid', 'test'],   # fields to which the fiald above will be splitted
    split_proportions=[0.7, 0.2, 0.1],  #proportions for splitting
    split_seed=23,  # seed for splitting dataset
    seed=42)  # seed for iteration over dataset

2022-02-03 10:09:53.169 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/drive/MyDrive/НИР/valid.csv file
2022-02-03 10:09:53.172 WARNING in 'deeppavlov.dataset_readers.basic_classification_reader'['basic_classification_reader'] at line 112: Cannot find /content/drive/MyDrive/НИР/test.csv file
2022-02-03 10:09:53.176 INFO in 'deeppavlov.dataset_iterators.basic_classification_iterator'['basic_classification_iterator'] at line 74: Splitting field <<train>> to new fields <<['train', 'valid', 'test']>>


In [ ]:
x_train, y_train = train_iterator.get_instances(data_type='train')

x_valid, y_valid = train_iterator.get_instances(data_type='valid')

x_test, y_test = train_iterator.get_instances(data_type='test')

print('train: ')
for x, y in list(zip(x_train, y_train))[:3]:
   print('x:', x)
   print('y:', y)
   print('=================')

print('validation: ')
for x, y in list(zip(x_valid, y_valid))[:3]:
   print('x:', x)
   print('y:', y)
   print('=================')



train: 
x: «Адепт национализма»: Захарова о главе Минздрава Украины 
y: 1
x: Тюльпанов обует водителей по закону
y: 1
x: «Троллили мою семью»: скандалисты зарубили бабушку и внуков
y: 1
validation: 
x: Так, чисто поржать, ибо пьян.. 1. Ничего личного .. то есть - выложить скрин моего аккаунта с пометкой латентный гомосексуалист - это - ничего личного? 2. Просто пруфы предоставляйте .. не буду, потому что лень. На правоту не претендую. Кому не нравится, может привести антипруф, или поставить минус и идти дальше. 3. Извинюсь. Даже не вопрос. Если был неправ .. (тут я немного грамматические ошибки поправил, но не суть.. сам пренебрегаю заглавными буквами, ибо пишу с телефона ..) а. Ценю попытку извиниться, не каждый на это способен.. но: б. Если был неправ .. то есть пикабушник, зная меня только по моим постам, сделал вполне конкретный вывод, что я латентный гомосексуалист , о чём не преминул известить всё сообщество.. и понял, что не прав, только после того, как получил но носу (согласно

In [ ]:
classifier_RB.preparation(x_train, y_train, x_valid, y_valid)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
all_valid_loss = []
all_train_loss = []
all_valid_acc = []
all_train_acc = []

all_valid_loss, all_valid_acc, all_train_loss, all_train_acc = classifier_RB.train('RuBert-tiny.pth')

Epoch 1/6
Train loss 0.13043805648878126 accuracy 0.9717142857142858
Val loss 0.1439917480199729 accuracy 0.977
----------
Epoch 2/6
Train loss 0.07019090991924376 accuracy 0.9858571428571429
Val loss 0.0912519502074756 accuracy 0.983
----------
Epoch 3/6
Train loss 0.03817159573145857 accuracy 0.9924285714285715
Val loss 0.11089896882217727 accuracy 0.984
----------
Epoch 4/6
Train loss 0.023152695645216357 accuracy 0.9955714285714287
Val loss 0.12203656184298871 accuracy 0.9825
----------
Epoch 5/6
Train loss 0.0133350104753099 accuracy 0.9975714285714287
Val loss 0.13032806202474695 accuracy 0.984
----------
Epoch 6/6
Train loss 0.007689365708436136 accuracy 0.9985714285714287
Val loss 0.13571681420481901 accuracy 0.984
----------


AttributeError: ignored

In [ ]:
 model_save_path = '/content/drive/MyDrive/НИР/models/'+'RuBert-tiny.pth'
 model_save_path

'/content/drive/MyDrive/НИР/models/RuBert-tiny.pth'

In [ ]:
predictions = [str(classifier_RB.predict(t)) for t in x_test]

In [ ]:
precision, recall, f1score = precision_recall_fscore_support(y_test, predictions)[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

In [ ]:
import matplotlib.pyplot as plt 

fig, axs = plt.subplots(2, 2)
axs[0][0].plot([1,2,3,4], all_train_loss)
axs[0][1].plot([1,2,3,4], all_valid_loss)
axs[1][0].plot([1,2,3,4], all_train_acc)
axs[1][1].plot([1,2,3,4], all_valid_acc)